In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

from selenium.webdriver.common.by import By
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
import os
import requests
import pandas as pd

In [1012]:
while True:
    # 크롬드라이버 실행
    driver = webdriver.Chrome()

    # 크롬 드라이버에 url 주소 넣고 실행
    driver.get('https://map.naver.com/p/search/%ED%99%8D%EB%8C%80%EB%A7%9B%EC%A7%91?c=14.00,0,0,0,dh')
    time.sleep(3)

    # 페이지가 완전히 로딩되도록 3초동안 기다림

    # Switch to the iframe
    iframe = driver.find_element(By.XPATH, '//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)

    # 가게명을 찾아 변수에 저장 (xpath 이용방식)
    shop_name_tab = driver.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/a/div/div/span[1]')
    time.sleep(3)

    shop_name = shop_name_tab.text
    shop_name_tab.click()
    time.sleep(1)

    shop_folder_path = os.path.join(path_folder, shop_name)

    # shop_name에서 공백 제거
    shop_name_without_spaces = shop_name.replace(" ", "")

    # 중복 여부 확인 및 출력
    if os.path.exists(shop_folder_path) or os.path.exists(os.path.join(path_folder, shop_name_without_spaces)):
        print("중복가게")
        driver.quit()
    else:
        print("중복아님")
        break

중복가게
중복아님


In [1013]:
driver.switch_to.default_content()

iframe = driver.find_element(By.XPATH, '//*[@id="entryIframe"]')
driver.switch_to.frame(iframe)

# 메타데이터를 찾아 변수에 저장 (xpath 이용방식)
shop_meta_tab = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]')
shop_meta = shop_meta_tab.text

menu_tabs = driver.find_elements(By.CSS_SELECTOR, 'div.flicking-camera  > a > span')

photo_idx = np.argmax(np.array([t.text for t in menu_tabs]) == '사진')
photo_menu = menu_tabs[photo_idx]
photo_menu.click()
time.sleep(3)

 # '내부' 탭으로 드래그 이동하기
source_element = driver.find_elements(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div > div > div > span > a')[2]
source_element

# Create an ActionChains object
actions = ActionChains(driver)

# Perform the drag to the left action
# Adjust the x_offset based on your requirement (negative values move to the left)
x_offset = -50  # Adjust as needed
actions.drag_and_drop_by_offset(source_element, x_offset, 0).perform()

In [1014]:
x_offset = -50  # Adjust as needed
actions.drag_and_drop_by_offset(source_element, x_offset, 0).perform()

In [1015]:
detail_tabs = driver.find_elements(By.CSS_SELECTOR, 'div.flicking-camera > span > a')

indoor_idx = np.argmax(np.array([t.text for t in detail_tabs]) == '내부')
indoor_menu = detail_tabs[indoor_idx]
indoor_menu.click()
time.sleep(3)

print(shop_meta)

상곱창곱창,막창,양
알림받기
방문자리뷰 252블로그리뷰 84


In [1016]:
# 이미지 저장

interior_thumbnail = driver.find_elements(By.CSS_SELECTOR, "a.place_thumb > img")

link_thumbnail = []

for img in interior_thumbnail:

    link_thumbnail.append(img.get_attribute('src')) 

# path_folder의 경로는 각자 저장할 폴더의 경로를 적어줄 것(ex.img_download)
path_folder = r"C:\Users\gani\sesac3\project1\img_download3_hongdae"

if not os.path.isdir(path_folder):
    os.mkdir(path_folder)

# 이미지 다운로드 및 메타데이터 수집
meta = []

for i, link in enumerate(link_thumbnail):
    response = requests.get(link)
    if response.status_code == 200:
        extension = os.path.splitext(link)[-1].lower()

        # shop_name에서 공백 및 특수 문자 제거하여 폴더명 생성
        folder_name = ''.join(e for e in shop_name if e.isalnum())

        folder_path = os.path.join(path_folder, folder_name)

        # Create folder if it doesn't exist
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)

        # 이미지 파일명 생성
        image_name = f'{shop_name}_{i}.{extension}'
        file_path = os.path.join(folder_path, image_name)

        meta.append((i, file_path, link))
        
        with open(file_path, 'wb') as file:
            file.write(response.content)

# 메타데이터를 DataFrame으로 변환 및 CSV 파일로 저장

meta.append((i, file_path, link, shop_meta))

meta_df = pd.DataFrame(meta, columns=['Index', 'File Path', 'Image Link', 'Shop Meta'])
meta_df.to_csv(os.path.join(path_folder, f'{folder_name}_meta.csv'), index=False)

driver.quit()  # 작업이 끝나면 창을닫는다.

In [1017]:
# CSV 파일 개수 세기
csv_file_count = len([file for file in os.listdir(path_folder) if file.endswith(".csv")])
  
print(f"CSV 파일 개수: {csv_file_count}")

CSV 파일 개수: 100
